In [1]:
from numpy import poly1d
from sympy import init_session
import numpy as np
from scipy.interpolate import lagrange
from numpy.polynomial.polynomial import Polynomial
init_session()
from tqdm import tqdm
from IPython.display import display
from itertools import product

IPython console for SymPy 1.6.2 (Python 3.7.3-64-bit) (ground types: python)

These commands were executed:
>>> from __future__ import division
>>> from sympy import *
>>> x, y, z, t = symbols('x y z t')
>>> k, m, n = symbols('k m n', integer=True)
>>> f, g, h = symbols('f g h', cls=Function)
>>> init_printing()

Documentation can be found at https://docs.sympy.org/1.6.2/



In [35]:
from sympy import *

In [2]:
def get_factor(number: int):
    factors = list(factorint(number).keys())
    factors.append(number)
    factors.append(1)
    factors.append(-1)
    factors.extend([-x for x in factors.copy()])
    return np.unique(np.array(factors))

def lagrange_for_integers(x, w):
    M = len(x)
    p = poly1d(np.int32(0))
    for j in range(M):
        pt = poly1d(w[j])
        for k in range(M):
            if k == j:
                continue
            fac = x[j]-x[k]
            if x[k] % fac == 0:
                pt *= poly1d([1, -x[k]]) / fac
            else:
                return None
        p += pt
    return p

In [ ]:
x, y, z, t = symbols('x y z t')
# исходный полином
# X = Poly(x ** 5 - x ** 4 - 2 * x ** 3 - 8 * x ** 2 + 6 * x - 1)
# X = Poly(random_poly(x, 10, 0, 2), x)
X = Poly(x ** 4 + x ** 3 + x ** 2 - x - 2)
display("Исходный полином")
display(X)
n = degree(X) // 2
# вектор точек, в которых необходимо
R = np.array(range(0,n+1))
# вектор делителей
factors = list()
# раскладываем множество делителей X (i), где i это число это от 0 до n
for i in tqdm(range(0, n+1)):
    t = X(i)
    factors.append(get_factor(t))
    factors[i].sort()
factors_len = 1
# считаем число вариантов которые необходимо перебрать
for f in factors:
    factors_len = f.shape[0] * factors_len
# и делим его на 2 чтобы не перебирать лишние комбинации
factors_len = factors_len // 2
print(factors_len)
# выполняем прямое произведение между членами вектора факторов
ys = product(*factors)
# и начинаем перебор
for i, y_i in tqdm(enumerate(ys), total=factors_len):
    # интерполяция по Лагранжу
    k = lagrange_for_integers(R, y_i)
    if k is None:
        continue
    LC = k.c[0]
    k = k / LC
    k = k.c.astype(np.int32)
    buf_poly = Poly(k, x, domain = ZZ)
    if degree(buf_poly) > 0:
        # если исходный полином делиться на интерполяционный без остатка то выводим его
        quo, rem =  X.div(buf_poly)
        # print(rem)
        if rem == 0:
            # сделать чтобы не было рациональных + рекурсивное разложение
            display("Интерполяционный полином:")
            display(buf_poly)
            display("Результат деления исходного на интерполяционный:")
            display(quo)
            display("Проверка:")
            display(quo * buf_poly)
            display("-------------------------")
    if i >= factors_len:
        break

In [81]:
def kroneker(queue_ : list, r : bool = False):
    X = queue_.pop()
    res = list()
    n = degree(X) // 2 # вектор точек, в которых необходимо
    R = np.array(range(0,n+1))     # вектор делителей
    factors = list()
    factors_deg = set()
    display("--------------------------------------")
    display("Исходный полином")
    display("--------------------------------------")
    display(X)
    # display("Значения f(x)")
    for n_s in tqdm(range(0, n+1)):
        val = X(n_s)
        # display(val)
        if val == 0:
            mn = Poly(x - val, x)
            res.append(mn)
            q, _  = X.div(mn)
            res.append(q)
        else:
            factors.append(get_factor(val))
            factors[n_s].sort()
    factors_len = 1
    # считаем число вариантов которые необходимо перебрать
    # display("Все факторы")
    for f in factors:
        factors_len = f.shape[0] * factors_len
        # display(f)

    # и делим его на 2 чтобы не перебирать лишние комбинации
    factors_len = factors_len // 2
    display(factors)
    print(factors_len)
    # выполняем прямое произведение между членами вектора факторов
    ys = product(*factors)
    # и начинаем перебор
    # for i, y in enumerate(list(ys)):
    #     print(y)
    #     if i == factors_len:
    #         print("----------------------------")
    #
    for i, y_i in tqdm(enumerate(ys), total=factors_len):
        # интерполяция по Лагранжу
        k = lagrange_for_integers(R, y_i)
        if k is None:
            continue
        LC = k.c[0]
        k = k / LC
        k = k.c.astype(np.int32)
        buf_poly = Poly(k, x, domain = ZZ)
        # display(y_i)
        buf_deg = degree(buf_poly, gen=x)
        # display(buf_poly)
        if buf_deg > 0 and buf_deg not in factors_deg:
            # если исходный полином делиться на интерполяционный без остатка то выводим его
            quo, rem =  X.div(buf_poly)
            # print(rem)
            if rem == 0:
                # display("Интерполяционный полином:")
                # display(buf_poly)
                # display("Результат деления исходного на интерполяционный:")
                # display(quo)
                # display("Проверка:")
                # display(quo * buf_poly)
                # display("-------------------------")
                factors_deg.add(degree_list(buf_poly)[0])
                factors_deg.add(degree_list(quo)[0])
                res.append((quo, buf_poly, y_i))
                if r:
                    queue_.append(quo)
                    queue_.append(buf_poly)
        # if i >= factors_len:
        #     break
    if len(res) != 0:
        display("Интерполяционный полином и Результат деления исходного на интерполяционный:")
        for buf, quo_, fac in res:
            display("Факторы")
            display(fac)
            display("Исходный полином:")
            display(X)
            display("Интерполяционный полином:")
            display(buf)
            display("Результат деления исходного на интерполяционный:")
            display(quo_)

    display("--------------------------------------")

In [82]:
# X = Poly(x ** 4 + x ** 3 + x ** 2 - x - 2)
# X = Poly(x ** 4  - 2 * x ** 2 + 1)
X = Poly(x ** 4  - 20000 * x ** 2 + 10 ** 8)
# X = Poly(x ** 2 - 2 * x + 1)
X = sqf_part(X)
# X = Poly(x ** 5- 2 * x ** 3 + 7*x ** 2 - 5*x + 1)
# X = Poly(x ** 5 - x ** 4 - 2* x ** 3 - 8 * x ** 2 + 6 * x - 1)
queue = list()
queue.append(X)
while len(queue) != 0:
    kroneker(queue, True)

'--------------------------------------'

'Исходный полином'

'--------------------------------------'

 40%|████      | 16/40 [00:16<00:25,  1.05s/it]


[array([-10000, -5, -2, -1, 1, 2, 5, 10000], dtype=object),
 array([-9999, -101, -11, -3, -1, 1, 3, 11, 101, 9999], dtype=object)]

40


KeyboardInterrupt: 

In [ ]:
np.array([1., 1.5, 7.2]).astype(np.int32)

In [30]:
def S(t1, t2):
    d = gcd(LM(t1), LM(t2))
    f = LM(t1) / d
    g = LM(t2) / d
    return g * t1 - t2 * f

# t1 = Poly(x ** 2 * y + x)
# t2 = Poly(x * y ** 2 - x * y)
# display(S(t1, t2))
t1 = Poly(x ** 2 * y + x)
t2 = Poly(x * y - x)
display(S(t1, t2))

In [20]:
groebner([t1, t2], x,y,order='grlex')

In [50]:
def sq_free(T):
    p = T
    res_list = list()
    display(p)
    while degree(p, gen=x) !=0:
        Rx = gcd(T, T.diff())
        Tx, _ = div(T, Rx)
        Vx = gcd(Rx, Tx)
        res, _ = div(Tx, Vx)
        p = res
        res_list.append(res)
    return res_list

In [80]:
# X = Poly(x ** 4 + x ** 3 + x ** 2 - x - 2)
X = Poly(x ** 2  - 2 * x + 1)
l = sq_free(X)
for i in l:
    display(l)

In [64]:
# X = Poly(x ** 4  - 20000 * x ** 2 + 10 ** 8)
# sqf_list(X)


In [78]:
factorint(10000)